In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
from scipy.sparse import csr_matrix

In [2]:
header=['user_id', 'item_id', 'rating', 'timestamp']
train_data = pd.read_csv('train.csv', sep=',', names=header)
test_data = pd.read_csv('test.csv', sep=',' , names=header)

matrix = train_data.values

train_rows = matrix[:,0].astype('int')
train_cols = matrix[:,1].astype('int')
train_rates = matrix[:,2]

rows = matrix[:,0].astype('int')
cols = matrix[:,1].astype('int')
rates = matrix[:,2]
user_num = len(set(rows))
movi_num = len(set(cols))

users = list(sorted(set(rows)))
movies = list(sorted(set(cols)))

for i in range(len(rows)):
    rows[i] = users.index(rows[i])
for i in range(len(cols)):
    cols[i] = movies.index(cols[i])

In [3]:
test_matrix = test_data.values
test_rows = test_matrix[:,0].astype('int')
test_cols = test_matrix[:,1].astype('int')
test_rates = test_matrix[:,2]

In [4]:
R = csr_matrix((rates, (rows, cols))).toarray()

In [5]:
err_train = []
err_test = []

In [6]:
def test_err(full):
    err = 0
    for i in range(len(test_rows)):
        err = err+abs(full[users.index(test_rows[i]),movies.index(test_cols[i])]-test_rates[i])
    return err/len(test_rows)
def train_err(full):
    err = 0
    for i in range(len(rows)):
        err = err+abs(full[users.index(train_rows[i]),movies.index(train_cols[i])]-train_rates[i])
    return err/len(train_rows)

In [7]:
def save_csv(itr,full):
    pre_rows = []
    pre_cols = []
    pre_rates = []
    for i in range(full.shape[0]):
        us = users[i]
        for j in range(full.shape[1]):
            mo = movies[j]
            pre_rows.append(us)
            pre_cols.append(mo)
            pre_rates.append(full[i,j])
    pre = np.array([np.array(pre_rows).T,np.array(pre_cols).T,np.array(pre_rates).T])
    pre = pre.T
    pd.DataFrame(pre).to_csv("pre_mse_"+str(itr)+".csv",index = False,header=False)

In [8]:
class MF():
    def __init__(self, R, K, alpha, beta, iterations):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

    def train(self):
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            #training_process.append((i, mse))
            print("Iteration: %d ; " % i )
            full = self.full_matrix()
            err_train.append(train_err(full))
            err_test.append(test_err(full))
            print('train error',err_train[-1])
            print('test error',err_test[-1])
            
            # save full matrix here:
            if i == 23:
            #    save_csv(int(mse),full)
                print("saved")
                
        return training_process

    def mse(self):
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r - prediction)
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

In [9]:
mf = MF(R, K=100, alpha=0.01, beta=0.001, iterations=24)

In [10]:
mf.train()

Iteration: 0 ; 
train error 0.6979810821859248
test error 0.7267366395723144
Iteration: 1 ; 
train error 0.6761505154408742
test error 0.7089848239396798
Iteration: 2 ; 
train error 0.6641553524362525
test error 0.699402049557554
Iteration: 3 ; 
train error 0.6559269796613058
test error 0.6953824328534617
Iteration: 4 ; 
train error 0.6493707349520543
test error 0.6898576250508595
Iteration: 5 ; 
train error 0.6443614885777558
test error 0.6888308967755732
Iteration: 6 ; 
train error 0.638294178191235
test error 0.685873949876968
Iteration: 7 ; 
train error 0.6341885748628401
test error 0.6855211071294873
Iteration: 8 ; 
train error 0.6269467366143195
test error 0.68362826015763
Iteration: 9 ; 
train error 0.6183178827147177
test error 0.6819743473953286
Iteration: 10 ; 
train error 0.6058958527705959
test error 0.6807177699435711
Iteration: 11 ; 
train error 0.5914498675621991
test error 0.6786439549763216
Iteration: 12 ; 
train error 0.5737569509949155
test error 0.6777882114975348
I

[]